In [35]:
import mne
import pandas as pd
import os
import numpy as np
import csv

In [32]:
bdf_file = 'subj04_session2'

bdf_file_path = os.path.join('eeg_data', 'bdf', f"{bdf_file}.bdf") 

# Read the BDF file
raw = mne.io.read_raw_bdf(bdf_file_path, preload=True)
print("Channels:", raw.ch_names)

Extracting EDF parameters from /Users/charles/Developer/all-joined/alljoined_preprocessing/eeg_data/bdf/subj04_session2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1784319  =      0.000 ...  3484.998 secs...
Channels: ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'Status']


In [28]:
events = mne.find_events(raw, stim_channel='Status', shortest_event=1)
print("events.shape:", events.shape)
print("events[:10]:", events[:10])
print("events[-10:]:", events[-10:])

Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
12719 events found on stim channel Status
Event IDs: [    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   151   152   153   154   155   156   157   158   159   160   161   162
   163   164   1

events.shape: (12719, 3)
events[:10]: [[  368 65536 65734]
 [  637     0   201]
 [ 1020     0     1]
 [ 1037     0    39]
 [ 1366     0   252]
 [ 1425     0     1]
 [ 1441     0    21]
 [ 1780     0   252]
 [ 1843     0     1]
 [ 1856     0    33]]
events[-10:]: [[1783066       0     252]
 [1783124       0      16]
 [1783137       0     102]
 [1783467       0     252]
 [1783525       0      16]
 [1783542       0      58]
 [1783876       0     252]
 [1783936       0      16]
 [1783949       0      16]
 [1784295       0     252]]


In [26]:
unique_event_ids = np.unique(events[:, 2])  # Extract unique event IDs
print(unique_event_ids)

'''
1. **198:** start
2. **199:** stop
3. **201:** train block
4. **202-217:** testing blocks
5. **START_RECORD = 198; % the trigger to start the EEG recording**
6. **STOP_RECORD = 199; % the trigger to stop the EEG recording**
7. **CORRECT_HIT = 254; % the trigger that records a correct hit to a catch stimulus**
8. **FALSE_ALARM = 253; % the trigger that records a false hit to an experimental stimulus**
9. **CORRECT_REJ = 252; % the trigger that records a correct no-key to an experiment stimulus**
10. **MISS = 251; % the trigger that records a false no-key to a catch stimulus**
'''

[    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   151   152   153   154   155   156   157   158   159   160   161   162
   163   164   165   166   201   202   203   204   205   206   207   208
   209   210   211   212   213   214   215   216   251   252   253   254
 65536 65734]


'\n1. **198:** start\n2. **199:** stop\n3. **201:** train block\n4. **202-217:** testing blocks\n5. **START_RECORD = 198; % the trigger to start the EEG recording**\n6. **STOP_RECORD = 199; % the trigger to stop the EEG recording**\n7. **CORRECT_HIT = 254; % the trigger that records a correct hit to a catch stimulus**\n8. **FALSE_ALARM = 253; % the trigger that records a false hit to an experimental stimulus**\n9. **CORRECT_REJ = 252; % the trigger that records a correct no-key to an experiment stimulus**\n10. **MISS = 251; % the trigger that records a false no-key to a catch stimulus**\n'

In [46]:
# Output our events into a .csv file
csv_file_path = os.path.join('eeg_data', 'before', f"{bdf_file}.csv") 

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(['onset', 'sig', 'code'])
    
    # Write the data rows
    for event in events:
        writer.writerow(event)

the cell below consolidates the triplet event codes in the raw csv file in a single event code in the parsed csv file. Oddballs are ignored. If an error is outputted, the raw csv file likely has a phantom event code, and will require manual inspection to remove it.

In [48]:
def create_new_row(row, img_idx):
    new_row = row.copy()
    new_row.iloc[2] = img_idx

    return new_row

raw_eeg_df = pd.read_csv(csv_file_path)
new_df = pd.DataFrame(columns=raw_eeg_df.columns)

i = 2
block_number = 1
block_trigger_offset = 150
error_count = 0

while i < raw_eeg_df.shape[0] - 2:
    
    # New block, increment block number and add a block code to new csv
    if raw_eeg_df.iloc[i, 2] == 65536:
        block_number += 1
        i+=3
        
    first_row_event_code = raw_eeg_df.iloc[i, 2]
    second_row_event_code = raw_eeg_df.iloc[i+1, 2]
    third_row_event_code = raw_eeg_df.iloc[i+2, 2]

    # 254 - Oddball - Correct - hit 
    if third_row_event_code == 254 and first_row_event_code == block_number+block_trigger_offset:
        pass 
    
    # # 251 - Oddball - Miss - no hit
    elif third_row_event_code == 251 and first_row_event_code == block_number+block_trigger_offset:
        pass
        
    # 252 - Correct - no hit  
    elif third_row_event_code == 252 and first_row_event_code == block_number:
        img_idx = ((block_number - 1) % 8) * 120 + second_row_event_code
        modified_row = create_new_row(raw_eeg_df.iloc[i], img_idx)
        new_df = new_df._append(modified_row, ignore_index=True)
        
    # 253 - False hit 
    elif third_row_event_code == 253 and first_row_event_code == block_number:
        img_idx = ((block_number - 1) % 8) * 120 + second_row_event_code
        modified_row = create_new_row(raw_eeg_df.iloc[i], img_idx)
        new_df = new_df._append(modified_row, ignore_index=True)

    else:
        if error_count < 10:
            error_count += 1
            print(f'Error in line {i-2}:', first_row_event_code, third_row_event_code)

    # Jump 3 rows for next trial
    i += 3

# Save data
final_output_path = csv_file_path.replace('before', 'after')
new_df.to_csv(final_output_path, index=False)
print('Saved to: ', final_output_path)

Saved to:  eeg_data/after/subj04_session2.csv
